# WA County EV + Charger Projection (Lower vs Upper Scenarios)
This notebook reproduces **all** code you wrote (no skipping), and adds step-by-step explanations.

**Inputs (your local files):**
- `supercharger in washington state.xls`
- `Population 2024 age 25 to 59.xlsx`
- `coordinates_output.xlsm`

**Outputs (saved to your Desktop):**
- One Excel file per county with a `Forecast` sheet + a `Charts` sheet containing embedded PNG charts.


## Step 0 — Import packages
You import: 
- **pandas/numpy** for data wrangling and math
- **matplotlib** to generate charts
- **openpyxl** to write to Excel and embed images
- **os/math** for paths and rounding logic


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from openpyxl import load_workbook
from openpyxl.drawing.image import Image


## Step 1 — Read the 3 Excel inputs
You load:
1) The Supercharger site list (filtered later by county)
2) A population table (age 25–59) by county (you use the `total` column)
3) A WA EV registration/coordinate file (filtered later by county)


In [ ]:
# --- Step 1: Read Excel files ---
super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")


## Step 2 — Set statewide policy adoption targets
These targets are used to create a piecewise adoption curve:
- 45% by 2030
- 60% by 2035
- 70% by 2040
- 95% by 2050


In [ ]:
# --- Step 3: Policy targets ---
policy_targets = {2030: 0.45, 2035: 0.60, 2040: 0.70, 2050: 0.95}


## KING COUNTY — Build lower/upper charger scenarios + adoption forecast
### What you do here
1) Filter each dataset down to **King County**
2) Compute baselines: population (25–59), existing chargers, existing EVs
3) Compute two charger targets:
   - **Lower** target = geographic coverage (area / circle coverage)
   - **Upper** target = population-based (`population_25_59 / 1500`)
4) Simulate yearly builds (2025–2050) for each scenario
5) Create 3 charts and embed them into the output Excel


In [ ]:
# --- Step 2: King County data ---
king_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "King County")]
king_residents = residents_df[residents_df["County"] == "King County"]
king_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "King")]

population_column = "total"
population_25_59 = king_residents[population_column].values[0]
num_chargers = len(king_super)
num_evs = len(king_ev)

print(f"King County population (25-59): {population_25_59}")
print(f"Existing Superchargers: {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 4: Lower and Upper Bound charger targets ---
king_county_area = 2307  # sq mi
radius_miles = 2
area_per_charger = np.pi * radius_miles**2
lower_chargers = int(np.ceil(king_county_area / area_per_charger))  # geo coverage (2-mile radius)

upper_base_target = population_25_59 / 1500  # 1 per 1500 residents (baseline upper bound)

print(f"Lower bound chargers (geo coverage): {lower_chargers}")
print(f"Original upper bound (1/1500 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- Lower Bound (constant build per year) ---
current_lower = num_chargers
total_lower_needed = lower_chargers - num_chargers
n_years = len(years)
yearly_build_lower = int(round(total_lower_needed / n_years))

for y in years:
    new_lower = yearly_build_lower
    if current_lower + new_lower > lower_chargers:
        new_lower = lower_chargers - current_lower
    current_lower += new_lower

    # EV adoption
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
        effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (2-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": new_lower,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

# --- Upper Bound (3-phase slowdown) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate * 1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate * 0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate * 0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "king_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Chart 1 — Total Chargers ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)
plt.axvspan(2025, 2030, color="green", alpha=0.1, label="Phase 1")
plt.axvspan(2030, 2040, color="yellow", alpha=0.1, label="Phase 2")
plt.axvspan(2040, 2050, color="orange", alpha=0.1, label="Phase 3")
plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("King County: EV Charger Projection (Lower Constant vs 3-Phase Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()

charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "king_total_chargers.png")
plt.savefig(charger_chart)
plt.close()

# --- Step 8: Chart 2 — NEW Dual X-Axis Chart ---
plt.figure(figsize=(12,6))

# Plot chargers (Y-axis)
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.xlabel("Year (2025–2050)")
plt.ylabel("Total Chargers")
plt.title("King County: Total Chargers vs Adoption Rate (Dual X-Axis)")

ax = plt.gca()

# --- TOP X-AXIS (MANUAL POSITIONS, NO TRANSFORM) ---
ax_top = ax.secondary_xaxis('top')
ax_top.set_xlabel("EV Adoption Rate")

years = forecast_df["Year"].values
rates = forecast_df["Adoption_Rate"].values

ax_top.set_xticks(years)
ax_top.set_xticklabels([f"{r:.2f}" for r in rates], rotation=45, ha='center', fontsize=8)

# 🔥 FIX THE GAP — anchor spine EXACTLY at top, no outward shift
ax_top.spines['top'].set_position(('axes', 1.0))     # ← key fix
ax_top.tick_params(axis='x', pad=2)                  # tighten padding

plt.grid(True)
plt.legend()

# Apply tight_layout only after axes are positioned
plt.tight_layout()

dual_chart_path = os.path.join(os.path.expanduser("~"), "Desktop", "king_chargers_vs_adoption_dual_axis.png")
plt.savefig(dual_chart_path)
plt.close()

# --- Step 9: Chart 3 — EV Adoption Rate ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)
plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("King County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()

adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "king_adoption_rate.png")
plt.savefig(adoption_chart)
plt.close()

# --- Step 10: Embed all charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")

ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(dual_chart_path), "A40")   # ← New Chart #2
ws.add_image(Image(adoption_chart), "A80")

wb.save(desktop_path)

print("\n✅ Projection complete with all 3 charts (including dual-axis Chart #2).")
print(f"Excel saved to: {desktop_path}")


## PIERCE COUNTY — Same framework with different assumptions
Differences you set for Pierce:
- Radius = **3 miles** for the lower-bound coverage
- Upper bound = **1 charger per 2,500 residents**
- Lower build uses `math.ceil()` so you always build whole chargers, then force the last year to hit the exact target.


In [ ]:
# --- Step 2: Pierce County data ---
pierce_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Pierce County")]
pierce_residents = residents_df[residents_df["County"] == "Pierce County"]
pierce_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Pierce")]

population_column = "total"
population_25_59 = pierce_residents[population_column].values[0]
num_chargers = len(pierce_super)
num_evs = len(pierce_ev)

print(f"Pierce County population (25-59): {population_25_59}")
print(f"Existing Superchargers (from Excel): {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 4: Lower and Upper Bound charger targets ---
pierce_county_area = 1790  # sq mi
radius_miles = 3
area_per_charger = np.pi * radius_miles**2  # 28.27 sq mi per charger
lower_chargers = int(np.ceil(pierce_county_area / area_per_charger))  # geo coverage (3-mile radius)

upper_base_target = population_25_59 / 2500  # 1 per 2500 residents

print(f"Lower bound chargers (geo coverage, 3-mile radius): {lower_chargers}")
print(f"Original upper bound (1/2500 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- LOWER BOUND (constant build per year, rounded up) ---
current_lower = num_chargers
total_lower_needed = max(0, lower_chargers - num_chargers)
n_years = len(years)
yearly_build_lower_exact = total_lower_needed / n_years

for i, y in enumerate(years):
    new_lower = math.ceil(yearly_build_lower_exact)

    if current_lower + new_lower > lower_chargers or i == len(years) - 1:
        new_lower = lower_chargers - current_lower

    current_lower += new_lower

    # EV adoption logic
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 10 * (y - 2025) * 0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030] - 0.10) * 0.5
        effective_adoption = start_rate + (policy_targets[2035] - start_rate) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (3-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": int(new_lower),
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

# --- UPPER BOUND (3-phase slowdown) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate * 1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate * 0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate * 0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030] - 0.10) / 5 * (y - 2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030]) / 5 * (y - 2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035]) / 5 * (y - 2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040]) / 10 * (y - 2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "pierce_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Chart 1 (Total Chargers) ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.axvspan(2025, 2030, color="green", alpha=0.1)
plt.axvspan(2030, 2040, color="yellow", alpha=0.1)
plt.axvspan(2040, 2050, color="orange", alpha=0.1)

plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Pierce County: EV Charger Projection (Lower vs Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()

charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "pierce_total_chargers.png")
plt.savefig(charger_chart)
plt.close()

# --- Step 8: Chart 2 (Dual X-Axis: Chargers vs Adoption Rate) ---
plt.figure(figsize=(12,6))

for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.xlabel("Year (2025–2050)")
plt.ylabel("Total Chargers")
plt.title("Pierce County: Total Chargers vs Adoption Rate (Dual X-Axis)")

ax = plt.gca()

# TOP AXIS (manual tick mapping)
ax_top = ax.secondary_xaxis('top')
ax_top.set_xlabel("EV Adoption Rate")

years = forecast_df["Year"].values
rates = forecast_df["Adoption_Rate"].values

ax_top.set_xticks(years)
ax_top.set_xticklabels([f"{r:.2f}" for r in rates], rotation=45, fontsize=8)

# FIX GAP: align top axis flush to plot edge
ax_top.spines['top'].set_position(('axes', 1.0))
ax_top.tick_params(axis='x', pad=2)

plt.grid(True)
plt.legend()
plt.tight_layout()

dual_chart = os.path.join(os.path.expanduser("~"), "Desktop", "pierce_chargers_vs_adoption_dual_axis.png")
plt.savefig(dual_chart)
plt.close()

# --- Step 9: Chart 3 (Adoption Rate Only) ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)

plt.xlabel("Year")
plt.ylabel("Adoption Rate")
plt.title("Pierce County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()

adop_chart = os.path.join(os.path.expanduser("~"), "Desktop", "pierce_adoption_rate.png")
plt.savefig(adop_chart)
plt.close()

# --- Step 10: Embed all charts in Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")

ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(dual_chart), "A40")
ws.add_image(Image(adop_chart), "A80")

wb.save(desktop_path)

print("\n✅ Pierce County projection complete with dual-axis Chart #2 added!")
print("Saved to:", desktop_path)


## KITSAP COUNTY — Same framework with different assumptions
Differences you set for Kitsap:
- Radius = **5 miles** for the lower-bound coverage
- Upper bound = **1 charger per 2,500 residents**
- Kitsap starts with **0** existing Superchargers in your dataset


In [ ]:
# --- Step 2: Kitsap County data ---
kitsap_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Kitsap County")]
kitsap_residents = residents_df[residents_df["County"] == "Kitsap County"]
kitsap_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Kitsap")]

population_column = "total"
population_25_59 = kitsap_residents[population_column].values[0]
num_chargers = len(kitsap_super)
num_evs = len(kitsap_ev)

print(f"Kitsap County population (25-59): {population_25_59}")
print(f"Existing Superchargers (from Excel): {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 4: Lower and Upper Bound charger targets ---
kitsap_county_area = 566  # sq mi
radius_miles = 5
area_per_charger = np.pi * radius_miles**2  # 78.54 sq mi per charger
lower_chargers = int(np.ceil(kitsap_county_area / area_per_charger))  # geo coverage (5-mile radius)

upper_base_target = population_25_59 / 2500  # 1 per 2500 residents

print(f"Lower bound chargers (geo coverage, 5-mile radius): {lower_chargers}")
print(f"Original upper bound (1/2500 residents): {upper_base_target:.0f}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
results = []

# --- LOWER BOUND (constant build per year) ---
current_lower = num_chargers
total_lower_needed = max(0, lower_chargers - num_chargers)
n_years = len(years)
yearly_build_lower_exact = total_lower_needed / n_years

for i, y in enumerate(years):
    new_lower = math.ceil(yearly_build_lower_exact)
    if current_lower + new_lower > lower_chargers or i == len(years)-1:
        new_lower = lower_chargers - current_lower
    current_lower += new_lower

    # EV adoption logic
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030]-0.10)/10 * (y-2025)*0.5
    elif y <= 2035:
        start_rate = 0.10 + (policy_targets[2030]-0.10)*0.5
        effective_adoption = start_rate + (policy_targets[2035]-start_rate)/5 * (y-2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040]-policy_targets[2035])/5 * (y-2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050]-policy_targets[2040])/10 * (y-2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Lower Bound (5-mile coverage, constant build)",
        "Year": y,
        "Total_Chargers": int(current_lower),
        "New_Chargers": int(new_lower),
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

# --- UPPER BOUND (3-phase slowdown) ---
base_build_rate = upper_base_target / n_years
current_upper = num_chargers

for y in years:
    if 2025 <= y <= 2030:
        yearly_add = int(round(base_build_rate*1.0))
    elif 2031 <= y <= 2040:
        yearly_add = int(round(base_build_rate*0.6))
    elif 2041 <= y <= 2050:
        yearly_add = int(round(base_build_rate*0.2))
    else:
        yearly_add = 0

    current_upper += yearly_add

    # EV adoption (upper)
    if y <= 2030:
        effective_adoption = 0.10 + (policy_targets[2030]-0.10)/5 * (y-2025)
    elif y <= 2035:
        effective_adoption = policy_targets[2030] + (policy_targets[2035]-policy_targets[2030])/5 * (y-2030)
    elif y <= 2040:
        effective_adoption = policy_targets[2035] + (policy_targets[2040]-policy_targets[2035])/5 * (y-2035)
    else:
        effective_adoption = policy_targets[2040] + (policy_targets[2050]-policy_targets[2040])/10 * (y-2040)

    evs = int(population_25_59 * effective_adoption)

    results.append({
        "Scenario": "Upper Bound (3-phase slowdown)",
        "Year": y,
        "Total_Chargers": int(current_upper),
        "New_Chargers": yearly_add,
        "EVs": evs,
        "Adoption_Rate": round(effective_adoption, 4)
    })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_county_ev_projection_constant_lower.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Chart 1 (Total Chargers) ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.axvspan(2025,2030,color="green",alpha=0.1)
plt.axvspan(2030,2040,color="yellow",alpha=0.1)
plt.axvspan(2040,2050,color="orange",alpha=0.1)

plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Kitsap County: EV Charger Projection (Lower Constant vs 3-Phase Upper)")
plt.legend()
plt.grid(True)
plt.tight_layout()

charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_charger_projection_constant_lower.png")
plt.savefig(charger_chart)
plt.close()

# --- Step 8: Chart 2 (Dual X-Axis: Chargers vs Adoption) ---
plt.figure(figsize=(12,6))

for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.xlabel("Year (2025–2050)")
plt.ylabel("Total Chargers")
plt.title("Kitsap County: Total Chargers vs Adoption Rate (Dual X-Axis)")

ax = plt.gca()

# TOP AXIS (manual mapping)
ax_top = ax.secondary_xaxis('top')
ax_top.set_xlabel("EV Adoption Rate")

years = forecast_df["Year"].values
rates = forecast_df["Adoption_Rate"].values

ax_top.set_xticks(years)
ax_top.set_xticklabels([f"{r:.2f}" for r in rates], rotation=45, fontsize=8)

# FIX GAP
ax_top.spines['top'].set_position(('axes', 1.0))
ax_top.tick_params(axis='x', pad=2)

plt.grid(True)
plt.legend()
plt.tight_layout()

dual_chart = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_chargers_vs_adoption_dual_axis.png")
plt.savefig(dual_chart)
plt.close()

# --- Step 9: Chart 3 (Adoption Rate Only) ---
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)

plt.xlabel("Year")
plt.ylabel("Adoption Rate")
plt.title("Kitsap County: EV Adoption Rate Forecast (2025–2050)")
plt.legend()
plt.grid(True)
plt.tight_layout()

adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "kitsap_ev_adoption_rate_constant_lower.png")
plt.savefig(adoption_chart)
plt.close()

# --- Step 10: Embed charts ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")

ws.add_image(Image(charger_chart), "A1")
ws.add_image(Image(dual_chart), "A40")
ws.add_image(Image(adoption_chart), "A80")

wb.save(desktop_path)

print("\n✅ Kitsap County projection complete with dual-axis Chart #2 added!")
print("Saved to:", desktop_path)


## CHELAN COUNTY — Two scenarios with different build rules
Differences you set for Chelan:
- Lower coverage uses radius = **15 miles** in the formula (per your code)
- Upper target uses `population_25_59 / 5000`
- Lower scenario builds **1 charger/year** until target is hit (front-loaded)
- Upper scenario spreads remaining chargers over remaining years using `ceil(remaining / remaining_years)`
- You generate **4 charts** for Chelan (Adoption, EVs, Chargers, Dual-axis)


In [ ]:
# --- Step 2: Chelan County data ---
chelan_super = super_df[(super_df["State"] == "Washington") & (super_df["County"] == "Chelan County")]
chelan_residents = residents_df[residents_df["County"] == "Chelan County"]
chelan_ev = ev_df[(ev_df["State"] == "WA") & (ev_df["County"] == "Chelan")]

population_column = "total"
population_25_59 = chelan_residents[population_column].values[0]
num_chargers = len(chelan_super)
num_evs = len(chelan_ev)

print(f"Chelan County population (25-59): {population_25_59}")
print(f"Existing Superchargers: {num_chargers}")
print(f"Existing EVs: {num_evs}")

# --- Step 4: Lower and Upper Bound charger targets ---
county_area = 2965  # sq mi
radius_miles = 15
lower_chargers = math.ceil(county_area / (3.1416 * radius_miles**2))  
upper_chargers = math.ceil(population_25_59 / 5000)

print(f"Lower bound chargers (geo coverage): {lower_chargers}")
print(f"Upper bound chargers (population-based): {upper_chargers}")

# --- Step 5: Build scenarios ---
years = list(range(2025, 2051))
scenarios = {
    "Lower Bound (20-mi radius, front-loaded, whole numbers)": {"target": lower_chargers, "start": num_chargers},
    "Upper Bound (10,000 res/charger)": {"target": upper_chargers, "start": num_chargers}
}

results = []

for name, params in scenarios.items():
    target_ch = params["target"]
    current_ch = params["start"]

    for y in years:

        # --- Charger build logic ---
        if "Lower" in name:
            if current_ch < target_ch:
                new_ch = 1
                if current_ch + new_ch > target_ch:
                    new_ch = target_ch - current_ch
                current_ch += new_ch
            else:
                new_ch = 0
        else:
            remaining = target_ch - current_ch
            remaining_years = 2050 - y + 1
            if remaining > 0:
                new_ch = math.ceil(remaining / remaining_years)
                if current_ch + new_ch > target_ch:
                    new_ch = target_ch - current_ch
                current_ch += new_ch
            else:
                new_ch = 0

        # --- Adoption logic ---
        if "Lower" in name:
            if y <= 2030:
                effective_adoption = 0.10 + (policy_targets[2030] - 0.10)/10 * (y - 2025) * 0.5
            elif y <= 2035:
                start_rate = 0.10 + (policy_targets[2030] - 0.10)*0.5
                effective_adoption = start_rate + (policy_targets[2035] - start_rate)/5 * (y - 2030)
            elif y <= 2040:
                effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035])/5 * (y - 2035)
            else:
                effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040])/10 * (y - 2040)
        else:
            if y <= 2030:
                effective_adoption = 0.10 + (policy_targets[2030] - 0.10)/5 * (y - 2025)
            elif y <= 2035:
                effective_adoption = policy_targets[2030] + (policy_targets[2035] - policy_targets[2030])/5 * (y - 2030)
            elif y <= 2040:
                effective_adoption = policy_targets[2035] + (policy_targets[2040] - policy_targets[2035])/5 * (y - 2035)
            else:
                effective_adoption = policy_targets[2040] + (policy_targets[2050] - policy_targets[2040])/10 * (y - 2040)

        evs = int(population_25_59 * effective_adoption)

        results.append({
            "Scenario": name,
            "Year": y,
            "Total_Chargers": int(current_ch),
            "New_Chargers": int(new_ch),
            "EVs": evs,
            "Adoption_Rate": round(effective_adoption, 4)
        })

forecast_df = pd.DataFrame(results)

# --- Step 6: Save results to Excel ---
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_county_ev_projection_2025_2050.xlsx")
with pd.ExcelWriter(desktop_path, engine="openpyxl") as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast", index=False)

# --- Step 7: Chart 1 (Adoption Rate) ---
adoption_chart = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_adoption_rate.png")
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Adoption_Rate"], marker="o", label=sc)

plt.xlabel("Year")
plt.ylabel("EV Adoption Rate")
plt.title("EV Adoption Rate by Scenario (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(adoption_chart)
plt.close()

# --- Step 8: Chart 2 (EV Registrations) ---
evs_chart = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_evs_by_year.png")
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["EVs"], marker="o", label=sc)

plt.xlabel("Year")
plt.ylabel("EV Registrations")
plt.title("EV Registrations by Year (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(evs_chart)
plt.close()

# --- Step 9: Chart 3 (Total Chargers) ---
charger_chart = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_total_chargers_by_year.png")
plt.figure(figsize=(10,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.xlabel("Year")
plt.ylabel("Total Chargers")
plt.title("Total Chargers by Year (Chelan County)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(charger_chart)
plt.close()

# --- Step 10: Chart 4 (Dual X-Axis: Chargers vs Adoption Rate) ---
dual_chart = os.path.join(os.path.expanduser("~"), "Desktop", "chelan_chargers_vs_adoption_dual_axis.png")

plt.figure(figsize=(12,6))
for sc in forecast_df["Scenario"].unique():
    df = forecast_df[forecast_df["Scenario"] == sc]
    plt.plot(df["Year"], df["Total_Chargers"], marker="o", label=sc)

plt.xlabel("Year (2025–2050)")
plt.ylabel("Total Chargers")
plt.title("Chelan County: Total Chargers vs Adoption Rate (Dual X-Axis)")

ax = plt.gca()

# Top axis (manual, aligned ticks)
ax_top = ax.secondary_xaxis('top')
ax_top.set_xlabel("EV Adoption Rate")

years = forecast_df["Year"].values
rates = forecast_df["Adoption_Rate"].values

ax_top.set_xticks(years)
ax_top.set_xticklabels([f"{r:.2f}" for r in rates], rotation=45, fontsize=8)

# Fix spacing (no gap)
ax_top.spines['top'].set_position(('axes', 1.0))
ax_top.tick_params(axis='x', pad=2)

plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(dual_chart)
plt.close()

# --- Step 11: Insert all 4 charts into Excel ---
wb = load_workbook(desktop_path)
ws = wb.create_sheet(title="Charts")

ws.add_image(Image(adoption_chart), "A1")
ws.add_image(Image(evs_chart), "A40")
ws.add_image(Image(charger_chart), "A80")
ws.add_image(Image(dual_chart), "A120")

wb.save(desktop_path)

print("\n✅ Chelan County EV Projection complete — Dual-axis chart added and Excel updated!")
print("Saved to:", desktop_path)
